In [2]:
API_TOKEN = '40e207e2-ca32-416f-a4f5-583f3049f55b'
URL_GRAPHQL = 'https://www.poktscan.com/api/graphql'


In [3]:
import asyncio
import json
from os import path
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url=URL_GRAPHQL, headers={'Authorization': API_TOKEN})


async def send_query(file_name: str, query: str, params: dict={}):
    async with Client(
        transport=transport, fetch_schema_from_transport=True) as session:
        query = gql(query)
        result = await session.execute(query, variable_values=params)
        
        file = open(path.join('./data', f'{file_name}.json'), 'w')
        file.write(json.dumps(result))
        file.close()
    

### Request to get rewards data

In [4]:
# Provide a GraphQL query
get_rewards_query = """
query getChainsRewards($from: String!, $to: String!, $timeSeriesAggregation: TimeSeriesAggregation, $interval: BigInt) {
  getChainsRewards(
    from: $from
    to: $to
    timeSeriesAggregation: $timeSeriesAggregation
    interval: $interval
  ) {
    first
    first_time
    last
    last_time
    total_relays
    total_pokt
    total_by_chain {
      chain
      total_relays
      total_pokt
      __typename
    }
    units {
      first
      last
      point_id
      point
      chains {
        chain
        total_relays
        total_pokt
        validators_avg
        earn_avg
        __typename
      }
      __typename
    }
    __typename
  }
}

"""

params = {
    "from": "2022-04-30",
    "to": "2022-05-31",
    "timeSeriesAggregation": "day"
}

loop = asyncio.get_running_loop()
task = loop.create_task(send_query("rewards",get_rewards_query, params=params))

### Nodes Geolocation data

In [5]:
# Provide a GraphQL query
get_geolocation_query = """
query{
  nodesGeoLocationData {
    items {
      country
      staked
      unstaking
      validators
    }
  }
}

"""

task = loop.create_task(send_query("nodes_geolocation",get_geolocation_query))

Task exception was never retrieved
future: <Task finished name='Task-4' coro=<send_query() done, defined at /tmp/ipykernel_3833/2098654156.py:11> exception=TimeoutError()>
Traceback (most recent call last):
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/site-packages/gql/transport/aiohttp.py", line 286, in execute
    async with self.session.post(self.url, ssl=self.ssl, **post_args) as resp:
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/site-packages/aiohttp/client.py", line 1138, in __aenter__
    self._resp = await self._coro
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/site-packages/aiohttp/client.py", line 559, in _request
    await resp.start(conn)
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 898, in start
    message, payload = await protocol.read()  # type: ignore[union-attr]
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/sit

### SendNodes Data

In [8]:
# Get sendnodes nodes data
get_sendnodes_data = """
query nodes($page: Int!, $search: String!, $limit: Int!,  $filter: String) {
  nodes(search: $search, page: $page, limit: $limit, filter: $filter) {
    pageInfo {
      total
      __typename
    }
    items {
      _id
      height
      parse_time
      producer
      public_key
      address
      balance
      status
      service_url
      service_domain
      tokens
      unstaking_time
      jailed
      chains
      missed_signing_blocks_counter
      index_offset
      jailed_blocks_counter
      jailed_until
      start_height
      __typename
    }
    __typename
  }
}
"""
domain = "sendnodes.io"
params = {
    "filter": f"[[\"service_domain\",\"contains\",\"{domain}\"],\"or\",[\"address\",\"contains\",\"{domain}\"],\"or\",[\"service_url\",\"contains\",\"{domain}\"],\"or\",[\"country\",\"contains\",\"{domain}\"],\"or\",[\"version\",\"contains\",\"{domain}\"]]",
    "page": 1,
    "limit": 100,
    "search": ""
}

task1 = loop.create_task(send_query("sendnodes_data_1",get_sendnodes_data, params=params))



In [9]:
# Getting data for second page
params['page'] = 2
task2 = loop.create_task(send_query(
    "sendnodes_data_2", get_sendnodes_data, params=params))


Task exception was never retrieved
future: <Task finished name='Task-25' coro=<send_query() done, defined at /tmp/ipykernel_3833/2098654156.py:11> exception=TransportAlreadyConnected('Transport is already connected')>
Traceback (most recent call last):
  File "/tmp/ipykernel_3833/2098654156.py", line 12, in send_query
    async with Client(
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/site-packages/gql/client.py", line 597, in __aenter__
    await self.transport.connect()
  File "/home/dry/workspace/sendnodes/pokt-stats/venv/lib/python3.9/site-packages/gql/transport/aiohttp.py", line 113, in connect
    raise TransportAlreadyConnected("Transport is already connected")
gql.transport.exceptions.TransportAlreadyConnected: Transport is already connected


### Pokt Earn Performance

In [10]:
pokt_earn_query = """ query{
    getPoktEarnPerformance{
        thirty_day_pokt_avg
        thirty_day_relays_avg
    }
}
"""
pokt_perf_perf = loop.create_task(send_query(
    "pokt_earn_perf", pokt_earn_query))


### Largest nodes Runners stats

In [ ]:
largest_nodes_runners_query = """query{
  largestNodeRunners{
    items{
      service_domain
      validators
      power
    }
    total_power
  }
}
"""

largest_nodes_runners_task = loop.create_task(send_query(
    "largest_nodes_runners", largest_nodes_runners_query))


### SendNodes Summary

gql.transport.exceptions.TransportQueryError("{'message': 'Cannot return null for non-nullable field Node.version.', 'locations': [{'line': 27, 'column': 7}], 'path': ['nodes', 'items', 98, 'version'], 'extensions': {'code': 'INTERNAL_SERVER_ERROR'}}")